# Question 6 (10 pts) -- COMPLETE
Show examples or situations when the use of kernel procedure is more efficient in terms of training and prediction computational time w.r.t. polynomial features transformation

interpeitation of that:

Sometimes data is not clearly seperatable. To be able to seperate it you need to manipulate the features to make a fairly ok seperation line. To do this you can always transform the features but if you transform each point, you can run into huge computational times. 

To avoid that issue you can use a kernel procedure such as gaussian kernel to speed up the process and allow for more complex categorizations.

In [21]:
# lets get some play data 
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
import time

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import precision_score, accuracy_score

from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier

iris = datasets.load_iris()

iris_X = iris.data

iris_y = iris.target


X = pd.DataFrame(iris_X, columns=iris.feature_names)
y = pd.DataFrame(iris_y, columns={"target"})

# split
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size = 0.4)

Basic implemenation with polynomial feature transformation using logistic regression classifier --NO KERNEL

In [2]:
def poly_linear_base(Xtrain, ytrain, Xtest, ytest):
    tic = time.perf_counter()
    train_x = Xtrain
    train_y = ytrain
    test_x = Xtest
    test_y = ytest
    reg_pipe = Pipeline([('standardize', StandardScaler()),
                         ('polynomial', PolynomialFeatures(degree=4, include_bias=False)),
                         ('classify', LogisticRegression())])

    reg_pipe.fit(train_x, train_y)
    ac = accuracy_score(test_y, reg_pipe.predict(test_x))

    toc = time.perf_counter()
    print(f"standard logic regression in {toc - tic:0.4f} seconds with accuracy of {ac}")


In [3]:
poly_linear_base(X_train, y_train, X_test, y_test)

standard logic regression in 0.0770 seconds with accuracy of 0.9666666666666667


Gaussian Kernel Implementation and time:


In [8]:
def kernel(Xtrain, ytrain, Xtest, ytest):
    tic = time.perf_counter()

    train_x = Xtrain
    train_y = ytrain
    test_x = Xtest
    test_y = ytest

    kernel = 1.0 * RBF(1.0)
    pc = GaussianProcessClassifier(kernel=kernel, random_state=0).fit(train_x, train_y)

    ac = accuracy_score(test_y, pc.predict(test_x))

    toc = time.perf_counter()
    print(f"Classificiation with kernel process in {toc - tic:0.4f} seconds with accuracy of {ac}")

In [9]:
kernel(X_train, y_train, X_test, y_test)

Classificiation with kernel process in 0.4373 seconds with accuracy of 0.9333333333333333


Notice how the kernel in this small data set does not preform as quickly but if we use a larger dataset then our kernel classifier will end up preforming faster than the generic poly feature logistic regression. Again this occurs because for each feature point we need to map it to space and that space can be infinite. If it is then doing the transformation ends up taking alot more time than using a kernel. The kernel allows the data to be mapped into latent space without having to transform its vectors.